## Início do Pipeline de Modelagem e Rastreamento com MLflow

Este notebook tem como objetivo iniciar o pipeline de experimentos, carregando os datasets finais da camada `Curated` para criar os conjuntos de treino e teste totalmente alinhados. O fluxo inclui a separação de variáveis preditoras (`X`) e variável alvo (`y`), além da configuração inicial do MLflow Tracking, garantindo que todos os parâmetros, métricas e artefatos do modelo sejam rastreados de forma coerente e versionável.


In [1]:
#  ETAPA: Carga dos Dados Curados e Configuração do MLflow Tracking

"""
Executa:
1) Validação do diretório de trabalho.
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'.
3) Separação de X_train, y_train, X_test, y_test.
4) Configuração do Tracking URI do MLflow.
"""

import os
import pandas as pd
import mlflow

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths coerentes
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\nTreino shape:", train_df.shape)
print("Teste shape:", test_df.shape)

# 4️⃣ Separar X e y
TARGET = 'Credit_Score_Standard'  # Ajuste para seu target real

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]

print("\nX_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

# 5️⃣ Configurar MLflow Tracking URI
mlflow.set_tracking_uri("http://mlflow:5000")  # Ajuste se necessário
print("\nTracking URI configurado:", mlflow.get_tracking_uri())


Current Working Directory: /workspace

Treino shape: (100000, 6305)
Teste shape: (50000, 6305)

X_train: (100000, 6304)
y_train: (100000,)
X_test: (50000, 6304)
y_test: (50000,)

Tracking URI configurado: http://mlflow:5000


## Experimento Baseline com MLflow e Monitoramento de Progresso

Nesta etapa será rodado o primeiro experimento baseline usando o MLflow para rastrear parâmetros, métricas e artefatos. Para acompanhar operações potencialmente demoradas, como o ajuste do modelo (`fit`) e a geração de métricas, será utilizado o `tqdm` para monitorar loops de forma explícita. Isso garante visibilidade do progresso em tempo real, além de manter a rastreabilidade completa do pipeline.


In [2]:
#  ETAPA: Refazer Baseline — Normalização do Working Directory, Carga Curated, Reconstrução de y e Tracking MLflow

"""
Executa:
1) Validação e normalização do diretório de trabalho (CWD).
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'.
3) Reconstrução de y a partir de colunas dummy.
4) Separação coerente de X e y.
5) Tracking URI e credenciais MinIO explícitos.
6) Treino DecisionTreeClassifier com barra de progresso.
7) Logging MLflow de hiperparâmetros, métricas e artefato.
8) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Validar e corrigir CWD
print("Current Working Directory (antes):", os.getcwd())
os.chdir('/workspace')
print("Current Working Directory (depois):", os.getcwd())

# 2️⃣ Paths absolutos coerentes
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\nTreino shape:", train_df.shape)
print("Teste shape:", test_df.shape)
print("\ntrain_df.head(5):")
print(train_df.head(5))

# 4️⃣ Reconstruir y a partir de colunas dummy
dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("\nColunas de classe detectadas:", dummy_cols)

# Reconstrói y_train
y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

# Reconstrói y_test
y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# 5️⃣ Tracking URI e credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

print("\nTracking URI:", mlflow.get_tracking_uri())
print("MLFLOW_S3_ENDPOINT_URL:", os.environ['MLFLOW_S3_ENDPOINT_URL'])

# 6️⃣ Cria/recupera experimento
experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="Baseline_DecisionTree_Curated") as run:
    params = {"max_depth": 5, "random_state": 42}
    mlflow.log_params(params)

    model = DecisionTreeClassifier(**params)

    print("\nTreinando modelo com barra de progresso:")
    for _ in tqdm(range(1), desc="Fitting model"):
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)
    mlflow.sklearn.log_model(model, "model")

    # ✅ Prints finais coerentes — SOMENTE com 127.0.0.1
    print(f"\nRun ID: {run.info.run_id}")
    print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View experiment at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}")


Current Working Directory (antes): /workspace/notebooks
Current Working Directory (depois): /workspace

Treino shape: (100000, 6305)
Teste shape: (50000, 6305)

train_df.head(5):
     Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0   23.0       19114.12            1824.843333                  3   
1   23.0       19114.12                    NaN                  3   
2 -500.0       19114.12                    NaN                  3   
3   23.0       19114.12                    NaN                  3   
4   23.0       19114.12            1824.843333                  3   

   Num_Credit_Card  Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
0                4              3                    3                   4.0   
1                4              3                   -1                   4.0   
2                4              3                    3                   4.0   
3                4              3                    5                   4.0   
4     

Fitting model: 100%|██████████| 1/1 [00:12<00:00, 12.90s/it]



Accuracy: 0.5496
F1 Score: 0.7093

Run ID: 86de72fd063e485fa584c1d8c0395aca
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run Baseline_DecisionTree_Curated at: http://mlflow:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
🧪 View experiment at: http://mlflow:5000/#/experiments/1


## ETAPA: Melhoria do Modelo com GridSearchCV e MLflow Tracking

Este bloco marca a transição do experimento baseline para uma etapa de otimização incremental, usando GridSearchCV para explorar múltiplas combinações de hiperparâmetros de forma sistemática e rastreada.

## Objetivo
- Encontrar a configuração de hiperparâmetros mais eficaz para a Árvore de Decisão (DecisionTreeClassifier).
- Registrar cada combinação testada como um run único no MLflow, com seus parâmetros, métricas e artefato final.

## Princípios aplicados
- Tracking URI e backend MinIO/S3 mantidos consistentes.
- Cada variação é rastreável, sem sobrescrever runs anteriores.
- Uso de tqdm para barra de progresso, garantindo visibilidade em loops demorados.
- Prints finais coerentes, com links 127.0.0.1 para acesso ao MLflow UI.

## Resultado esperado
- Métricas comparáveis entre baseline e grid search.
- Melhor modelo salvo como artefato no bucket MinIO.
- Próximo passo: preparar o pipeline para registrar o modelo validado no Registry do MLflow.


In [4]:
# 🔧 ETAPA: GridSearch Manual com ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Itera ParameterGrid manualmente.
5) Barra tqdm avança por combinação.
6) Loga cada combinação como run separado no MLflow.
7) Prints coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid manual
param_grid = {
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5, 10]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual...")

for params in tqdm(grid, desc="Runs"):
    with mlflow.start_run(run_name=f"GridSearch_Manual_{params}") as run:
        model = DecisionTreeClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual...


Runs:  11%|█         | 1/9 [00:15<02:06, 15.75s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 8347a501625844da89aaa891510faa55
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  22%|██▏       | 2/9 [00:27<01:34, 13.56s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 5}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 9f9ded00e4594c7197535ae840bc8bfc
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 7, 'min_samples_split': 2}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: 9c9d83b3de294b5aab2357b42a67aa06
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  89%|████████▉ | 8/9 [01:35<00:11, 11.69s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 5}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: d2685d32bdcb42b783e358095e9f5b63
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs: 100%|██████████| 9/9 [01:46<00:00, 11.88s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 10}
Accuracy: 0.5424 | F1 Score: 0.7033
Run ID: 0cfc26f47de549969c8fdb2ceef4565c
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 10} at: http://mlflow:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [5]:
# 🔧 ETAPA: GridSearch Manual com RandomForest, ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Executa GridSearch manual com RandomForestClassifier.
5) Usa ParameterGrid + tqdm para barra real por combinação.
6) Loga cada run no MLflow com credenciais MinIO coerentes.
7) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid com RandomForest
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "min_samples_split": [2, 5]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual RandomForest...")

for params in tqdm(grid, desc="Runs RandomForest"):
    with mlflow.start_run(run_name=f"GridSearch_RF_{params}") as run:
        model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual RandomForest...


Runs RandomForest:  12%|█▎        | 1/8 [00:10<01:14, 10.66s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 0.1160 | F1 Score: 0.2078
Run ID: 7e1e8f44f2a14b469b513b183d4a0ae5
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  25%|██▌       | 2/8 [00:18<00:55,  9.26s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.0560 | F1 Score: 0.1061
Run ID: 7e2a7fd6532a415790f7e0297eaef518
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.1582 | F1 Score: 0.2732
Run ID: 3a35386b08854259abb939bffb29abb8
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  88%|████████▊ | 7/8 [00:54<00:07,  7.32s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50}
Accuracy: 0.2117 | F1 Score: 0.3494
Run ID: ee503b2c1af948db890c83f7a178a5de
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest: 100%|██████████| 8/8 [01:02<00:00,  7.78s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy: 0.1581 | F1 Score: 0.2730
Run ID: 4693946ae8bf47a799823ab6e1844b93
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [1]:
# 🔧 ETAPA: GridSearch Manual com GradientBoostingClassifier, ParameterGrid, tqdm e MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega 'train_clean.csv' da camada processed.
3) Separa X e y com target original.
4) Aplica pd.get_dummies() no X.
5) Usa train_test_split com stratify.
6) Executa GridSearch manual com GradientBoostingClassifier.
7) Barra tqdm para progresso real.
8) Loga cada run separadamente no MLflow.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Carregar dados
df = pd.read_csv('data/processed/train_clean.csv')
print("\ndf shape:", df.shape)
print("\ndf.head(5):\n", df.head(5))

# 3️⃣ Separa X e y
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# 4️⃣ Pré-processa X igual curated
X = pd.get_dummies(X)

# 5️⃣ train_test_split + alinhamento coerente
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

# 6️⃣ Configurar ParameterGrid
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1]
}
grid = ParameterGrid(param_grid)

# 7️⃣ Tracking MLflow + MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual — GradientBoostingClassifier...")

for params in tqdm(grid, desc="Runs GBoost"):
    with mlflow.start_run(run_name=f"GridSearch_GBoost_{params}") as run:
        model = GradientBoostingClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace/notebooks
CWD depois: /workspace

df shape: (100000, 28)

df.head(5):
        ID Customer_ID     Month           Name    Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh   23.0  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh   23.0  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh -500.0  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh   23.0  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh   23.0  821-00-0265  Scientist   

   Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0       19114.12            1824.843333                  3  ...     Unknown   
1       19114.12                    NaN                  3  ...        Good   
2       19114.12                    NaN                  3  ...        Good   
3       19114.12                    NaN                  3  ...        Good   
4       19114.12         

: 

#  Diagnóstico do Footprint de Memória

Antes de rodar fitting, esta célula calcula o uso real de memória dos DataFrames `X_train` e `y_train`.  
Garante rastreabilidade sobre quanta RAM o kernel irá consumir, considerando `deep=True` para contar objetos, ponteiros e índices.  
Este valor deve ser menor que 70% da RAM real do container para evitar travamento por OOM Killer.


## Recarga dos Datasets Curated — Caminho corrigido

Esta célula recarrega os datasets `train_curated.csv` e `test_curated.csv` usando caminho relativo correto, garantindo coerência com a estrutura `/workspace/`.



In [3]:
# ETAPA: Recarga dos Datasets Curated

import pandas as pd

train_df = pd.read_csv("../data/curated/train_curated.csv")
test_df = pd.read_csv("../data/curated/test_curated.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")


Train shape: (100000, 6305)
Test shape: (50000, 6305)


In [7]:
print(train_df.columns.tolist())


['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Loan_Bin', 'Changed_Credit_Limit_Bin', 'Num_of_Delayed_Payment_Bin', 'Credit_History_Age_Bin', 'Month_Num', 'Occupation_Architect', 'Occupation_Developer', 'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Entrepreneur', 'Occupation_Journalist', 'Occupation_Lawyer', 'Occupation_Manager', 'Occupation_Mechanic', 'Occupation_Media_Manager', 'Occupation_Musician', 'Occupation_Scientist', 'Occupation_Teacher', 'Occupation_Unknown', 'Occupation_Writer', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Credit-Builder Loan, Credit-Builder Loan, Mortgage Loan, and Personal Loan', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Student Loan, and Student Loan', 'Type_of_Loan_Auto Loan, Auto Loan, A

---
REABRIR O FEATURE_ENGINEERING_CURADORIA.IPNYB PARA DIMINUIR CARDINALIDADE
---

---
